# **Reto II**

### 1. Dataset

Los datos de origen son proporcionados en un archivos csv:

* udfs: dataset con datos de operaciones financieras.

### 2. Columnas y significado:

* nb: número de referencia de la operación.
* contract: identificador de contrato.
* udf_ref: identificador de operación de trading.
* fmly: familia a la que pertenece la operación financiera.
* grp: grupo al que pertenece la operación financiera.
* type: tipo de operación financiera.
* country: país de origen de la operación.
* udf_name: campo informado en el registro.
* num_value: valor numérico.
* string_value: valor de cadena de caracteres.
* date_value: valor de fecha.
* data_timestamp_part: marca temporal.
* data_date_part: fecha en la que se almacena la información.
* source_system: fuente de los datos.

### 3. Descripción del problema:

Si hacemos una visión general a nuestro conjunto de datos, podemos observar como hay hasta 10 registros (filas) para cada valor de *nb*, donde cada registro solo da información para un valor de *udf_name*. Esto es un gasto innecesario de almacenamiento y computación, además de complicar los futuros cálculos derivados de estos datos. Por esta razón, necesitamos convertir estos registros con el mismo *nb* a un solo registro.

Nuestro dataframe final tendrá que contener las siguientes columnas: `nb, M_CCY, M_CLIENT, M_CRDTCHRG, M_DIRECTIAV, M_DISCMARGIN, M_LIQDTYCHRG, M_MVA, M_RVA, M_SELLER, M_SUCURSAL`

* nb: debe contener el número de referencia de la operación.
* M_CLIENT, M_SELLER, M_CCY, M_SUCURSAL: deben mapear el valor de *string_value*
* M_DISCMARGIN, M_DIRECTIAV, M_LIQDTYCHRG, M_CRDTCHRG, , M_MVA, M_RVA: deben mapear el valor de *num_value*


Una vez tengamos este resultado, necesitaremos eliminar las operaciones que no tengan informados ninguno de los siguientes campos:

M_DISCMARGIN, M_DIRECTIAV, M_LIQDTYCHRG, M_CRDTCHRG, M_MVA, M_RVA, M_SELLER

No informados en este caso significa que o son valores nulos, vacíos o 0, en el caso de los campos numéricos.

### 4. Reto:

* Obtener un dataframe final que contenga las columnas indicadas, con un registro por *nb* y con los valores correctos mapeados.
* Las operaciones con los campos M_DISCMARGIN, M_DIRECTIAV, M_LIQDTYCHRG, M_CRDTCHRG, , M_MVA, M_RVA, M_SELLER no informados no deben existir.
* Hacerlo de la manera más eficiente posible a nivel computacional.

**NOTA:** Cada uno de los pasos descritos en el problema pueden efectuarse en una sola línea.

### Inicialización de SparkSession:

In [14]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = (SparkSession
 .builder
 .appName("Reto 2")
 .master("local")
 .getOrCreate())

In [10]:
udfs = (spark.read.format("csv")
        .option("header", "true")
        .option("delimiter", ";")
        .load("./data/udfs.csv")
        .cache()
       )

### Resultado:

**INSTRUCCIONES**: El DataFrame resultante debe almacenarse en la variable `resultado`, sustituyendo el valor `None` por el código que consideréis oportuno. De esta forma podréis comprobar si el resultado es correcto.

In [75]:
resultado = (udfs
             .select("nb",
                     "udf_name",
                     "string_value",
                     "num_value")
             .groupBy("nb")
             .pivot("udf_name")
             .agg(F.first("string_value"),F.first("num_value"))
             .select(F.col("nb"),
                     F.col("M_CCY_first(string_value)").alias("M_CCY"),  
                     F.col("M_CLIENT_first(string_value)").alias("M_CLIENT"), 
                     F.col("M_CRDTCHRG_first(num_value)").alias("M_CRDTCHRG"), 
                     F.col("M_DIRECTIAV_first(num_value)").alias("M_DIRECTIAV"),
                     F.col("M_DISCMARGIN_first(num_value)").alias("M_DISCMARGIN"), 
                     F.col("M_LIQDTYCHRG_first(num_value)").alias("M_LIQDTYCHRG"), 
                     F.col("M_MVA_first(num_value)").alias("M_MVA"), 
                     F.col("M_RVA_first(num_value)").alias("M_RVA"),
                     F.col("M_SELLER_first(string_value)").alias("M_SELLER"),
                     F.col("M_SUCURSAL_first(string_value)").alias("M_SUCURSAL")
                    )              
             .where( (F.col("M_DISCMARGIN") > 0) |
                     (F.col("M_DIRECTIAV") > 0) |
                    (F.col("M_LIQDTYCHRG") > 0) |
                    (F.col("M_CRDTCHRG") > 0) |
                    (F.col("M_MVA") > 0) |
                    (F.col("M_RVA") > 0) |
                    (F.length(F.col("M_SELLER")) > 0)
                   ).cache()
            )
udfs.unpersist()

DataFrame[nb: string, contract: string, udf_ref: string, fmly: string, grp: string, type: string, country: string, udf_name: string, num_value: string, string_value: string, date_value: string, data_timestamp_part: string, data_date_part: string, source_system: string]

In [76]:
assert(len(resultado.columns) == 11)
assert(resultado.columns[4] == "M_DIRECTIAV")
assert(resultado.select("M_SELLER").filter(F.col("nb") == 23037162).first()[0] == "AMAM")
assert(resultado.select("M_SELLER").filter(F.col("nb") == 19665186).first()[0] == "LB_VSTAVRE")
assert(resultado.select("M_RVA").filter(F.col("nb") == 444111222).first()[0] == "8956")

In [80]:
resultado.toPandas()

,nb,M_CCY,M_CLIENT,M_CRDTCHRG,M_DIRECTIAV,M_DISCMARGIN,M_LIQDTYCHRG,M_MVA,M_RVA,M_SELLER,M_SUCURSAL
0,20513130,None,CCMO,0.000000000000,0.000000000000,10,0.000000000000,20,0.000000000000,WATT,5493
1,18710605,MXN,None,None,0.000000000000,None,None,None,None,AMAM,None
2,20533916,None,CCMO,0.000000000000,0.000000000000,50,0.000000000000,30,0.000000000000,WATT,1999
3,23097010,EUR,None,None,0.000000000000,None,None,None,None,AMAM,None
4,555111222,USD,None,30.000000000000,0.000000000000,10,50.000000000000,20,0.000000000000,LB_TLECLER,1999
5,22784054,EUR,None,None,0.000000000000,None,None,None,None,AMAM,None
6,22798005,EUR,None,None,0.000000000000,None,None,None,None,AMAM,None
7,22853656,BRL,None,None,0.000000000000,None,None,None,None,AMAM,None
8,10000001,None,NULL,20,0.000000000000,10,30,0.000000000000,0.000000000000,SELLER1,1999
9,19665186,None,NULL,20.000000000000,1200.000000000000,100,20.000000000000,0.000000000000,0.000000000000,LB_VSTAVRE,5493


In [79]:
resultado.unpersist()

DataFrame[nb: string, M_CCY: string, M_CLIENT: string, M_CRDTCHRG: string, M_DIRECTIAV: string, M_DISCMARGIN: string, M_LIQDTYCHRG: string, M_MVA: string, M_RVA: string, M_SELLER: string, M_SUCURSAL: string]